<a href="https://colab.research.google.com/github/vpilone/CMPSC497-Midterm/blob/main/CMPSC497_Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install datasets

import tensorflow as tf
from tensorflow.keras import models, layers, utils

In [2]:
from datasets import load_dataset

trainDataset = load_dataset("tomaarsen/MultiCoNER", "en", split="train")
# trainDataset = trainDataset.with_format("tf")
testDataset = load_dataset("tomaarsen/MultiCoNER", "en", split="test")
# testDataset = testDataset.with_format("tf")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
vocabToIndexDict = {}
indexToVocabDict = {}
curIndex = 1

trainingSentances = []
vocabWords = []

for sentance in trainDataset[:]['tokens']:
  trainingSentances.append(sentance)
  for word in sentance:
    if word not in vocabWords:
      vocabWords.append(word)
    if vocabToIndexDict.get(word) == None:
      vocabToIndexDict[word] = curIndex
      indexToVocabDict[curIndex] = word
      curIndex +=1

In [4]:
!pip install gensim

In [5]:
from gensim.models.word2vec import Word2Vec

w2vModel = Word2Vec(trainDataset[:]['tokens'], min_count=1, window=4, vector_size=50,negative=3)
w2vModel.add_null_word()

In [6]:
print(w2vModel.wv.distance('his','he'))
print(w2vModel.wv.distance('queen','his'))

0.020865559577941895
0.04645603895187378


In [7]:
unknownVocab = []
sentanceEmbedded = []
sentanceEmbedded.append(tf.stack([[0]*50]))
totalEmbbeded = []
batch = 0
for sentance in trainDataset[:]['tokens']:
  batch += 1
  wordEmbedded = []
  for word in sentance:
    try:
      # wordEmbedded.append(w2vModel.wv[word])
      wordEmbedded.append(w2vModel[word])
    except:
      unknownVocab.append(word)
      # wordEmbedded.append([1]*w2vModel.wv.vector_size)
      wordEmbedded.append([1]*w2vModel.vector_size)
  sentanceEmbedded.append(tf.stack(wordEmbedded))
paddedEmbedded = utils.pad_sequences(sentanceEmbedded, padding="post")
totalEmbbeded=(tf.stack(paddedEmbedded))
#print(sentanceEmbedded)

In [8]:
#version for Sparse Categorical Error
newEncodedData = []
newEncodedData.append(tf.stack([[0]*50]))

for newSentance in trainDataset[:]['tokens']:
  newEncodedSentance = []
  for newWord in newSentance:
    try:
      #newEncodedSentance.append(w2vModel.wv[word])
      newEncodedSentance.append(w2vModel[newWord])
    except:
      #newEncodedSentance.append([0]*w2vModel.wv.vector_size)
      newEncodedSentance.append([1]*w2vModel.vector_size)
  newEncodedData.append(tf.stack(newEncodedSentance))
newPaddedData = utils.pad_sequences(newEncodedData, padding="post", dtype=float)

print(newPaddedData.shape)

(15301, 41, 50)


In [9]:
newTags = []
newTags.append([0]*50)

for newTag in trainDataset[:]['ner_tags']:
  newTags.append(newTag)
newPaddedTags = utils.pad_sequences(newTags, padding="post")
newStackedPaddedTags = tf.stack(newPaddedTags)

In [10]:
from tensorflow.keras import losses as losses
from tensorflow.keras import metrics as mts

newRNNModel = models.Sequential()
newRNNModel.add(layers.SimpleRNN(units=50))
newRNNModel.add(layers.Dense(units=50))
# newRNNModel.add(layers.)
newRNNModel.compile(loss=losses.CosineSimilarity(), metrics=['accuracy'])
#newRNNModel.compile(loss=losses.cosine_similarity, metrics=[mts.Accuracy])
newRNNModel.fit(newPaddedData, newPaddedTags, epochs=5)
# # print(type(w2vModel.wv['test']))
# # print(inputs)
# # print(output)

Epoch 1/5
479/479 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0807 - loss: -0.3542
Epoch 2/5
479/479 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1233 - loss: -0.4358
Epoch 3/5
479/479 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.1523 - loss: -0.4486
Epoch 4/5
479/479 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1618 - loss: -0.4500
Epoch 5/5
479/479 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1746 - loss: -0.4591


In [11]:
#version for Sparse Categorical Error
newEncodedTestData = []
newEncodedTestData.append(tf.stack([[0]*50]))

for newTestSentance in testDataset[:2000]['tokens']:
  newEncodedTestSentance = []
  for newTestWord in newTestSentance:
    try:
      newEncodedTestSentance.append(w2vModel.wv[newTestWord])
      #newTestSentance.append(w2vModel[newTestWord])
    except:
      #newEncodedSentance.append([1]*w2vModel.wv.vector_size)
      newEncodedTestSentance.append([1]*w2vModel.vector_size)
  newEncodedTestData.append(tf.stack(newEncodedTestSentance))
newPaddedTestData = utils.pad_sequences(newEncodedTestData, padding="post", dtype=float)

print(newPaddedTestData.shape)

(2001, 36, 50)


In [12]:
newTestTags = []
newTestTags.append([0]*50)

for newTestTag in testDataset[:2000]['ner_tags']:
  newTestTags.append(newTestTag)
newPaddedTestTags = utils.pad_sequences(newTestTags, padding="post")
newStackedPaddedTestTags = tf.stack(newPaddedTestTags)

print(newStackedPaddedTestTags.shape)

(2001, 50)


In [13]:
results = newRNNModel.evaluate(newPaddedTestData, newPaddedTestTags)
print("Loss: " ,results[0])
print("Accuracy: " ,results[1]*100, "%")

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0686 - loss: -0.3917
Loss:  -0.3908470571041107
Accuracy:  7.446277141571045 %
